In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import psycopg2
from sklearn.metrics import confusion_matrix
import seaborn as sns
from config import db_password
import time

In [2]:
# Read in the country_vaccinations CSV file as Pandas DataFrames.
human_dev_index = pd.read_excel('../Resources/human_dev_index.xlsx',index=True)
human_dev_index 


,HDI rank,Country,Human Development Type,Human Development Index (HDI) 2019,Life expectancy at birth 2019,Expected years of schooling 2019,Mean years of schoolingn 2019,Gross national income (GNI) per capita 2019,GNI per capita rank minus HDI rank 2019,HDI rank 2018
0,1,Norway,Very High,0.957,82.40,18.06615,12.897750,66494.252170,7,1
1,2,Ireland,Very High,0.955,82.31,18.70529,12.666331,68370.587370,4,3
2,2,Switzerland,Very High,0.955,83.78,16.32844,13.380812,69393.520760,3,2
3,4,"Hong Kong, China (SAR)",Very High,0.949,84.86,16.92947,12.279960,62984.765530,7,4
4,4,Iceland,Very High,0.949,82.99,19.08309,12.772787,54682.380570,14,4
...,...,...,...,...,...,...,...,...,...,...
190,0,Monaco,Other,0.000,0.00,0.00000,0.000000,0.000000,0,0
191,0,Nauru,Other,0.000,0.00,11.23418,0.000000,16237.481010,0,0
192,0,San Marino,Other,0.000,0.00,13.04993,0.000000,0.000000,0,0
193,0,Somalia,Other,0.000,57.40,0.00000,0.000000,0.000000,0,0


In [3]:
# Change column name to match the SQL schema

human_dev_index = human_dev_index.rename({'Country' : 'country_name'}, axis =1)
human_dev_index 

,HDI rank,country_name,Human Development Type,Human Development Index (HDI) 2019,Life expectancy at birth 2019,Expected years of schooling 2019,Mean years of schoolingn 2019,Gross national income (GNI) per capita 2019,GNI per capita rank minus HDI rank 2019,HDI rank 2018
0,1,Norway,Very High,0.957,82.40,18.06615,12.897750,66494.252170,7,1
1,2,Ireland,Very High,0.955,82.31,18.70529,12.666331,68370.587370,4,3
2,2,Switzerland,Very High,0.955,83.78,16.32844,13.380812,69393.520760,3,2
3,4,"Hong Kong, China (SAR)",Very High,0.949,84.86,16.92947,12.279960,62984.765530,7,4
4,4,Iceland,Very High,0.949,82.99,19.08309,12.772787,54682.380570,14,4
...,...,...,...,...,...,...,...,...,...,...
190,0,Monaco,Other,0.000,0.00,0.00000,0.000000,0.000000,0,0
191,0,Nauru,Other,0.000,0.00,11.23418,0.000000,16237.481010,0,0
192,0,San Marino,Other,0.000,0.00,13.04993,0.000000,0.000000,0,0
193,0,Somalia,Other,0.000,57.40,0.00000,0.000000,0.000000,0,0


In [4]:
# Format column name as accepeted by SQL
human_dev_index = human_dev_index.rename({'Gross national income (GNI) per capita 2019' : 'Gross national income GNI per capita 2019'},axis =1)
human_dev_index  

,HDI rank,country_name,Human Development Type,Human Development Index (HDI) 2019,Life expectancy at birth 2019,Expected years of schooling 2019,Mean years of schoolingn 2019,Gross national income GNI per capita 2019,GNI per capita rank minus HDI rank 2019,HDI rank 2018
0,1,Norway,Very High,0.957,82.40,18.06615,12.897750,66494.252170,7,1
1,2,Ireland,Very High,0.955,82.31,18.70529,12.666331,68370.587370,4,3
2,2,Switzerland,Very High,0.955,83.78,16.32844,13.380812,69393.520760,3,2
3,4,"Hong Kong, China (SAR)",Very High,0.949,84.86,16.92947,12.279960,62984.765530,7,4
4,4,Iceland,Very High,0.949,82.99,19.08309,12.772787,54682.380570,14,4
...,...,...,...,...,...,...,...,...,...,...
190,0,Monaco,Other,0.000,0.00,0.00000,0.000000,0.000000,0,0
191,0,Nauru,Other,0.000,0.00,11.23418,0.000000,16237.481010,0,0
192,0,San Marino,Other,0.000,0.00,13.04993,0.000000,0.000000,0,0
193,0,Somalia,Other,0.000,57.40,0.00000,0.000000,0.000000,0,0


In [5]:
# Format column name as accepeted by SQL
human_dev_index = human_dev_index.rename({'Human Development Index (HDI) 2019' : 'Human Development Index HDI 2019'},axis =1)
human_dev_index  

,HDI rank,country_name,Human Development Type,Human Development Index HDI 2019,Life expectancy at birth 2019,Expected years of schooling 2019,Mean years of schoolingn 2019,Gross national income GNI per capita 2019,GNI per capita rank minus HDI rank 2019,HDI rank 2018
0,1,Norway,Very High,0.957,82.40,18.06615,12.897750,66494.252170,7,1
1,2,Ireland,Very High,0.955,82.31,18.70529,12.666331,68370.587370,4,3
2,2,Switzerland,Very High,0.955,83.78,16.32844,13.380812,69393.520760,3,2
3,4,"Hong Kong, China (SAR)",Very High,0.949,84.86,16.92947,12.279960,62984.765530,7,4
4,4,Iceland,Very High,0.949,82.99,19.08309,12.772787,54682.380570,14,4
...,...,...,...,...,...,...,...,...,...,...
190,0,Monaco,Other,0.000,0.00,0.00000,0.000000,0.000000,0,0
191,0,Nauru,Other,0.000,0.00,11.23418,0.000000,16237.481010,0,0
192,0,San Marino,Other,0.000,0.00,13.04993,0.000000,0.000000,0,0
193,0,Somalia,Other,0.000,57.40,0.00000,0.000000,0.000000,0,0


In [6]:
# Drop null values
human_dev_index = human_dev_index.dropna()
human_dev_index

,HDI rank,country_name,Human Development Type,Human Development Index HDI 2019,Life expectancy at birth 2019,Expected years of schooling 2019,Mean years of schoolingn 2019,Gross national income GNI per capita 2019,GNI per capita rank minus HDI rank 2019,HDI rank 2018
0,1,Norway,Very High,0.957,82.40,18.06615,12.897750,66494.252170,7,1
1,2,Ireland,Very High,0.955,82.31,18.70529,12.666331,68370.587370,4,3
2,2,Switzerland,Very High,0.955,83.78,16.32844,13.380812,69393.520760,3,2
3,4,"Hong Kong, China (SAR)",Very High,0.949,84.86,16.92947,12.279960,62984.765530,7,4
4,4,Iceland,Very High,0.949,82.99,19.08309,12.772787,54682.380570,14,4
...,...,...,...,...,...,...,...,...,...,...
190,0,Monaco,Other,0.000,0.00,0.00000,0.000000,0.000000,0,0
191,0,Nauru,Other,0.000,0.00,11.23418,0.000000,16237.481010,0,0
192,0,San Marino,Other,0.000,0.00,13.04993,0.000000,0.000000,0,0
193,0,Somalia,Other,0.000,57.40,0.00000,0.000000,0.000000,0,0


In [7]:
# Create the path to your file directory and variable for the file
human_dev_index  = human_dev_index.to_csv('../Resources/Cleaned_data/human_dev_index.csv',index =False)